# Silver To Gold Transformation (Level 2)

In this level of transformation the data is usually curated based on business logic. Since this project just focuses on migrating the data no business logic is applied here. Instead modification of columns is taken place, where ***white spaces(" ")*** in column names are replaced by an ***underscore("_")***.

In [ ]:
# Displays the files & folder of SalesLT folder in silver container.
dbutils.fs.ls("/mnt/silver/SalesLT")

In [ ]:
# Listing the files in gold container which will be empty for now.
dbutils.fs.ls("/mnt/gold")

In [ ]:
table_names = []

for i in dbutils.fs.ls("/mnt/silver/SalesLT"):
    table_names.append(i.name.split('/')[0])


In order to remove whitespaces in column names for all the tables, We iterate through each table one by one *load* it as dataframe and *replace* column names by iterating over the columns of each dataframe.


While iterating through the columns of each dataframe(table), A variable called ***new_column*** is created which stores the new column name by removing white spaces and replacing them with an underscore. 

Then ***spark.withCoulumnRenamed()*** funtion is used to replace the ***old_column*** name (containing white spaces) with ***new_column*** name (replaced with underscores).

Once all the columns names of a dataframe are transformed, the df is stored onto gold container in the corresponding folder. This process is repeated until the names of every table column are changed.

In [ ]:
from pyspark.sql.functions import col, regexp_replace

for table in table_names: # Iterating over the tables.
    table_path = "/mnt/silver/SalesLT/" + table
    df = spark.read.format("delta").load(table_path)
    columns = df.columns

    for old_column in columns: # Iterating over the columns of a table.
        new_column = "".join(["_" + char if char.isupper() and not old_column[i-1].isupper() else char for i, char in enumerate(old_column)]).lstrip("_")
        df = df.withColumnRenamed(old_column, new_column)
    
    output_path = "/mnt/gold/SalesLT/" + table + "/" # Output path to store the dataframe in gold container.
    df.write.format("delta").mode("overwrite").save(output_path) # Loading the data onto gold container.